## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

### Data cleaning
*By Kaylee Mo*

To start we had to merge our 3 datasets

1. We first decided to left merge the Tutition and Barron's Selectivity datasets with the Top Colleges dataset. We decided to do left merge because Top Colleges had the most colleges we were interested in. 

2. We then ran into a problem where we couldn't left merge with the typical pandas left merge because the college names were not exact. For example, UC Berkeley was also University of California, Berkeley in another dataset. We thus had to import FuzzyMatcher. Fuzzymatcher takes the 2 college names and based on their similarity, creates a score. Fuzzymatcher then uses a threshold, and if that score is above the threshold, the 2 college names are merged together. 

3. Lastly, there were many variables in the final dataset. We did initial cleaning where we dropped the variables where we knew we were not interested in. For example, we did not need the description of the colleges, the phone number and website of the college, and more. 

The code below implements the above cleaning.

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy.ma as ma
import numpy as np
import fuzzymatcher
import statsmodels.formula.api as sm

top_colleges_list = pd.read_csv('top_colleges_2022.csv')
barrons = pd.read_csv('Barrons College List.csv')
tuition = pd.read_csv('tuition_cost.csv')
tuition = tuition.drop(['state', 'state_code', 'type', 'degree_length',
       'room_and_board', 'in_state_tuition', 'in_state_total', 'out_of_state_total'], axis = 1)
barrons.columns = barrons.columns.str.rstrip()
barrons['College Name'] = barrons['College Name'].astype(str)
barrons['College Name'] = barrons['College Name'].apply(lambda x: x.strip())
data = fuzzymatcher.fuzzy_left_join(top_colleges_list, barrons, ['organizationName'], ['College Name'])
data = data.drop(['best_match_score', '__id_left', '__id_right','description', 'longitude', 'latitude', 'website', 'phoneNumber', 'carnegieClassification', 'studentPopulation', 'College Name', 'City and State'], axis = 1)
data2 = fuzzymatcher.fuzzy_left_join(data, tuition, ['organizationName'], ['name'])
data = data2.drop(['best_match_score', '__id_left', '__id_right','yearFounded', 'stateCode', 'name'], axis = 1)

### Data quality check
*By Kaylee Mo*

The code below visualizes the distribution of all the variables in the dataset, and their association with the response.

In [5]:
#Distribution of variables
data[['medianBaseSalary', 'rank', 'studentFacultyRatio', 'undergradPop', 'out_of_state_tuition', 'Selectivity', 'totalGrantAid']].describe()

,medianBaseSalary,rank,studentFacultyRatio,undergradPop,out_of_state_tuition,Selectivity,totalGrantAid
count,491.000000,498.000000,498.000000,498.000000,498.000000,498.000000,4.950000e+02
mean,116382.077393,249.485944,14.120482,12075.550201,36441.395582,2.255020,2.491186e+07
std,17161.769465,143.899350,5.203074,12628.161452,13053.807507,0.803178,1.936898e+07
min,77300.000000,1.000000,3.000000,421.000000,4105.000000,1.000000,2.307030e+05
25%,104300.000000,125.250000,10.000000,2613.750000,25159.000000,2.000000,1.219794e+07
50%,112800.000000,249.500000,13.500000,6844.500000,36480.500000,2.000000,1.904380e+07
75%,125000.000000,373.750000,17.000000,18655.250000,48431.750000,3.000000,3.218453e+07
max,173700.000000,498.000000,49.000000,84202.000000,59985.000000,3.000000,1.575583e+08


In [9]:
# Distribution of our 1 categorical variable Selectivity (values of 1,2,3 for most selective to least selective)
data.Selectivity.value_counts()

3.0    233
2.0    141
1.0    107
Name: Selectivity, dtype: int64

In [6]:
#Association of the response with the predictors

data[['medianBaseSalary', 'rank', 'studentFacultyRatio', 'undergradPop', 'out_of_state_tuition', 'Selectivity', 'totalGrantAid']].corr()

,medianBaseSalary,rank,studentFacultyRatio,undergradPop,out_of_state_tuition,Selectivity,totalGrantAid
medianBaseSalary,1.000000,-0.645747,-0.442872,-0.158305,0.584428,-0.494202,0.201261
rank,-0.645747,1.000000,0.102519,-0.289547,-0.391347,0.419108,-0.460018
studentFacultyRatio,-0.442872,0.102519,1.000000,0.605834,-0.659826,0.310886,0.132181
undergradPop,-0.158305,-0.289547,0.605834,1.000000,-0.415883,0.109013,0.597566
out_of_state_tuition,0.584428,-0.391347,-0.659826,-0.415883,1.000000,-0.435413,0.174126
Selectivity,-0.494202,0.419108,0.310886,0.109013,-0.435413,1.000000,-0.114388
totalGrantAid,0.201261,-0.460018,0.132181,0.597566,0.174126,-0.114388,1.000000


### Data Preparation
*By Kaylee Mo*

Lastly, to prep the data for model development, we checked the null values of the variables we were interested in. As you can see below, there were only 7 numbers missing max for a variable, which we found to be a relatively small number compared to the total number of observations. We dropped all null values and then started model development.

The code below implements the checking and dropping of null values.

In [7]:
# checking amount of null values
data[['medianBaseSalary', 'rank', 'studentFacultyRatio', 'undergradPop', 'out_of_state_tuition', 'Selectivity', 'totalGrantAid']].isnull().sum(axis=0)

medianBaseSalary        7
rank                    0
studentFacultyRatio     0
undergradPop            0
out_of_state_tuition    0
Selectivity             0
totalGrantAid           3
dtype: int64

In [8]:
data = data.dropna(axis=0)

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

In [10]:
X = data[['Selectivity', 'studentFacultyRatio', 'undergradPop', 'rank', 'out_of_state_tuition', 'totalGrantAid']]
X.columns[1:]
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
X = add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

vif_data

,feature,VIF
0,const,87.007709
1,Selectivity,1.415238
2,studentFacultyRatio,2.987181
3,undergradPop,3.991629
4,rank,1.899697
5,out_of_state_tuition,3.074411
6,totalGrantAid,2.366419


## Developing the model

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

### Code fitting the final model

Put the code(s) that fit the final model(s) in separate cell(s), i.e., the code with the `.ols()` or `.logit()` functions.

## Conclusions and Recommendations to stakeholder(s)

You may or may not have code to put in this section. Delete this section if it is irrelevant.